<div align="Center">

# JK Lakshmipat University
## Institute of Engineering and Technology
### Machine Learning (CS1138)
#### Project-I
#### RFM model-based Customer Segmentation using Clustering and Classification

</div>
<hr>

#### Importing the Libraries

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

In [2]:
sns.set_palette('icefire')
sns.set_style('darkgrid')

<hr>

### Data Configuration

#### Importing the Data

In [3]:
df1 = pd.read_excel('online_retail_II.xlsx', sheet_name='Year 2009-2010')
df2 = pd.read_excel('online_retail_II.xlsx', sheet_name='Year 2010-2011')
df = pd.concat([df1, df2])

#### Initial Dataset

In [4]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1067371 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 73.3+ MB


In [6]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,1.067371e+06,1067371,1.067371e+06,824364.000000
mean,9.938898e+00,2011-01-02 21:13:55.394028544,4.649388e+00,15324.638504
min,-8.099500e+04,2009-12-01 07:45:00,-5.359436e+04,12346.000000
25%,1.000000e+00,2010-07-09 09:46:00,1.250000e+00,13975.000000
50%,3.000000e+00,2010-12-07 15:28:00,2.100000e+00,15255.000000
75%,1.000000e+01,2011-07-22 10:23:00,4.150000e+00,16797.000000
max,8.099500e+04,2011-12-09 12:50:00,3.897000e+04,18287.000000
std,1.727058e+02,NaN,1.235531e+02,1697.464450


#### Imputing the Dataset

In [7]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

- Description : Not Available
- Customer ID : -1

In [20]:
df['Description'] = df['Description'].fillna('Not Available')
df['Customer ID'] = df['Customer ID'].fillna(-1)

In [9]:
df.drop_duplicates(keep='first', inplace=True)

#### Feature Engineering

In [10]:
df['Customer ID'] = df['Customer ID'].astype(int)
df['TotalPrice'] = df['Price'] * df['Quantity']

In [11]:
df['Country'] = df['Country'].astype('category')
df['Description'] = df['Description'].astype('category')

In [12]:
df[df['Price'] < 0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,-53594.36,-1,United Kingdom,-53594.36
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,-44031.79,-1,United Kingdom,-44031.79
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,-38925.87,-1,United Kingdom,-38925.87
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,-1,United Kingdom,-11062.06
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,-1,United Kingdom,-11062.06


In [13]:
df['Cancelled'] = df['Invoice'].astype(str).str.contains('C').astype(int)
df['Bad Debt'] = df['Invoice'].astype(str).str.contains('A').astype(int)
df['Invoice'] = df['Invoice'].astype(str).str.replace('[A-Z]', '', regex=True).astype(int)

StockCode contains Codes for different Situations, so it Cannot be Converted to numerical.

In [14]:
len(df['StockCode'].str.extractall(r"([a-zA-Z]+)").groupby(level=0).sum(numeric_only=False)[0].unique())

624

#### Final Dataset

In [15]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice,Cancelled,Bad Debt
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4,0,0
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,0,0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,0,0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8,0,0
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0,0,0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1033036 entries, 0 to 541909
Data columns (total 11 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1033036 non-null  int32         
 1   StockCode    1033036 non-null  object        
 2   Description  1033036 non-null  category      
 3   Quantity     1033036 non-null  int64         
 4   InvoiceDate  1033036 non-null  datetime64[ns]
 5   Price        1033036 non-null  float64       
 6   Customer ID  1033036 non-null  int32         
 7   Country      1033036 non-null  category      
 8   TotalPrice   1033036 non-null  float64       
 9   Cancelled    1033036 non-null  int32         
 10  Bad Debt     1033036 non-null  int32         
dtypes: category(2), datetime64[ns](1), float64(2), int32(4), int64(1), object(1)
memory usage: 66.2+ MB


In [17]:
df.describe()

,Invoice,Quantity,InvoiceDate,Price,Customer ID,TotalPrice,Cancelled,Bad Debt
count,1.033036e+06,1.033036e+06,1033036,1.033036e+06,1.033036e+06,1.033036e+06,1.033036e+06,1.033036e+06
mean,5.375749e+05,1.007688e+01,2011-01-03 14:30:35.429549824,4.613980e+00,1.182711e+04,1.825254e+01,1.849306e-02,5.808123e-06
min,4.894340e+05,-8.099500e+04,2009-12-01 07:45:00,-5.359436e+04,-1.000000e+00,-1.684696e+05,0.000000e+00,0.000000e+00
25%,5.145580e+05,1.000000e+00,2010-07-05 11:38:00,1.250000e+00,1.252600e+04,3.750000e+00,0.000000e+00,0.000000e+00
50%,5.380530e+05,3.000000e+00,2010-12-09 13:34:00,2.100000e+00,1.450600e+04,9.920000e+00,0.000000e+00,0.000000e+00
75%,5.614730e+05,1.000000e+01,2011-07-27 13:17:00,4.150000e+00,1.635900e+04,1.770000e+01,0.000000e+00,0.000000e+00
max,5.815870e+05,8.099500e+04,2011-12-09 12:50:00,3.897000e+04,1.828700e+04,1.684696e+05,1.000000e+00,1.000000e+00
std,2.694597e+04,1.751976e+02,NaN,1.223975e+02,6.592054e+03,2.956873e+02,1.347260e-01,2.409999e-03


<hr>

### Exploratory Data Analysis

#### Top Selling Products

#### Most Frequently Sold Products

#### Average No. of Orders per Customer

#### Average No. of Unique Items per Customer and per Order

#### Top Countries by No. of Customers and No. of Orders

#### Total Sales per Month, per Week and per Day

#### Cancelled Items Analysis

#### Bad Debt Analysis

<hr>

## Machine Learning

#### Spliting Data into Train, Test and Validate

In [18]:
dfShuffled = df.sample(frac=1, random_state=42)

In [19]:
dfTrain, dfTest = train_test_split(dfShuffled, test_size=0.2, random_state=1)

dfTrain, dfValidate = train_test_split(dfTrain, test_size=0.2, random_state=1)

<hr>

### RFM Analysis

<hr>

### BG/NBD CLV Modelling
Beta-Geometric/Negative Binomial Distribution Customer Lifetime Value Modelling

<hr>

### Gamma-Gamma Modelling

<hr>

### k-Means Clustering

<hr>

### Hierarchical Clustering

<hr>

### k-NN Classification

<hr>

### Logistic Regression

<hr>

## Conclusion

<hr>